# Import libraries

In [ ]:
import pandas as pd
import io
import requests
import PyPDF2
import spacy
import string
import re
from PyPDF2 import PdfReader
import os
import numpy as np

# 1. Download and read pdf

In [ ]:
# Folder to store pdf files
path = r"C:\Users\Hi\OneDrive - University of Bristol\SustainabilityReportAnalysis\WorkingSpace\Elaina\Report"


Import the links to download reports

In [ ]:
esg_sample = pd.read_excel(r"C:\Users\Hi\OneDrive - University of Bristol\SustainabilityReportAnalysis\WorkingSpace\Elaina\task.xlsx", sheet_name="overview_2016", usecols=range(6)).iloc[201:301]

In [ ]:
esg_report_link = pd.read_excel(r"C:\Users\Hi\OneDrive - University of Bristol\SustainabilityReportAnalysis\WorkingSpace\Elaina\task.xlsx")
esg_report_link["Symbol"] = esg_report_link["Ticker"].str[:-2]
esg_report_link["year"] = esg_report_link["Year"].str[-4:]
esg_report_link["URL_check"] = np.where(esg_report_link["Link"].isna(), esg_report_link["URL"], esg_report_link["Link"])
esg_report_link = esg_report_link[
                                  (esg_report_link["year"].isin(["2016", "2017", "2018", "2019", "2020", "2021"]))
                                 #& (esg_report_link["Note"] != 2)
                                  ]

In [ ]:
esg_report_link = esg_report_link[(esg_report_link["Symbol"].isin(esg_sample["Symbol"]))
                                  & (esg_report_link["year"].isin(["2016", "2017", "2018", "2019", "2020", "2021"]))
                                  & (esg_report_link["Note"] == 1)
                                  ]

In [ ]:
count = esg_report_link.groupby("Symbol").count()

Download pdf from link and save files in folder

In [ ]:
def download_pdf(df, folder_path):
    #df['status'] = False

    for index, row in df.iterrows():
        ticker = row['Symbol']
        year = row['Year']
        url = row['Link']

        # Skip if the URL is NaN or empty
        if pd.isna(url) or not url:
            print(f"Invalid URL for Ticker: {ticker}, Year: {year}")
            continue

        # Create the folder if it doesn't exist
        os.makedirs(folder_path, exist_ok=True)

        # Create a file path with the folder and file name
        file_path = os.path.join(folder_path, f'{ticker}_{year}.pdf')

        # Download the PDF file
        try:
            response = requests.get(url)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Error occurred while downloading the PDF for Ticker: {ticker}, Year: {year}")
            continue

        # Save the PDF file
        with open(file_path, 'wb') as file:
            file.write(response.content)

        #df.at[index, 'status'] = True
        print(f"PDF downloaded for Ticker: {ticker}, Year: {year}")

    print("PDF download completed.")

In [ ]:
download_pdf(esg_report_link[esg_report_link["Note"] != 2], path)

PDF downloaded for Ticker: GD, Year: FY2021
Error occurred while downloading the PDF for Ticker: GD, Year: FY2020
PDF downloaded for Ticker: GD, Year: FY2019
PDF downloaded for Ticker: GD, Year: FY2018
Invalid URL for Ticker: GD, Year: FY2017
Invalid URL for Ticker: GD, Year: FY2016
PDF downloaded for Ticker: GE, Year: FY2021
PDF downloaded for Ticker: GE, Year: FY2020
PDF downloaded for Ticker: GE, Year: FY2019
PDF downloaded for Ticker: GE, Year: FY2018
Invalid URL for Ticker: GE, Year: FY2017
Invalid URL for Ticker: GE, Year: FY2016
PDF downloaded for Ticker: GEN, Year: FY2021
PDF downloaded for Ticker: GEN, Year: FY2020
PDF downloaded for Ticker: GEN, Year: FY2019
PDF downloaded for Ticker: GEN, Year: FY2018
Error occurred while downloading the PDF for Ticker: GEN, Year: FY2017
Error occurred while downloading the PDF for Ticker: GEN, Year: FY2016
PDF downloaded for Ticker: GILD, Year: FY2021
PDF downloaded for Ticker: GILD, Year: FY2020
PDF downloaded for Ticker: GILD, Year: FY201

In [ ]:
download_pdf(esg_report_link[esg_report_link["Note"] == 2], path)

Error occurred while downloading the PDF for Ticker: GD, Year: FY2020
Error occurred while downloading the PDF for Ticker: HII, Year: FY2021
PDF downloaded for Ticker: HII, Year: FY2020


In [ ]:
download_pdf(esg_report_link.iloc[12:17], path)

PDF downloaded for Ticker: GEN, Year: FY2020
PDF downloaded for Ticker: GEN, Year: FY2019
PDF downloaded for Ticker: GEN, Year: FY2018
PDF downloaded for Ticker: GEN, Year: FY2017
PDF downloaded for Ticker: GEN, Year: FY2016
PDF download completed.


In [ ]:
esg_report_link.iloc[12:17]

,Ticker,Year,Filename_mannual,Filename,check_year,check_pdf,URL,Note,Link,Unnamed: 9,Symbol,year,URL_check
25,GEN.A,FY2020,GEN.A-FY2020,2020 Corporate Responsibility Report,0.0,59.0,https://www.nortonlifelock.com/content/dam/nor...,1.0,https://www.nortonlifelock.com/content/dam/nor...,NaN,GEN,2020,https://www.nortonlifelock.com/content/dam/nor...
26,GEN.A,FY2019,GEN.A-FY2019,2019 Corporate Responsibility Report,0.0,112.0,https://www.nortonlifelock.com/content/dam/nor...,1.0,https://www.responsibilityreports.com/HostedDa...,NaN,GEN,2019,https://www.responsibilityreports.com/HostedDa...
27,GEN.A,FY2018,GEN.A-FY2018,2018 Corporate Responsibility Report,0.0,100.0,https://www.symantec.com/content/dam/symantec/...,1.0,https://www.responsibilityreports.com/HostedDa...,NaN,GEN,2018,https://www.responsibilityreports.com/HostedDa...
28,GEN.A,FY2017,GEN.A-FY2017,Corporate Responsibility Report 2017,0.0,100.0,https://www.symantec.com/content/dam/symantec/...,1.0,https://www.responsibilityreports.com/HostedDa...,NaN,GEN,2017,https://www.responsibilityreports.com/HostedDa...
29,GEN.A,FY2016,GEN.A-FY2016,Corporate Sustainability Report 2016,0.0,110.0,https://www.symantec.com/content/dam/symantec/...,1.0,https://www.responsibilityreports.com/HostedDa...,NaN,GEN,2016,https://www.responsibilityreports.com/HostedDa...


In [ ]:
requests.get("https://www.responsibilityreports.com/HostedData/ResponsibilityReportArchive/i/NYSE_IFF_2021.pdf")

<Response [200]>

In [ ]:
response = requests.get("https://www.key.com/content/dam/kco/documents/businesses___institutions/2021_KeyCorp_ESG_Report.pdf")
response.raise_for_status()
response

KeyboardInterrupt: 

In [ ]:
try:
    response = requests.get(url)
    response.raise_for_status()
except requests.exceptions.RequestException as e:
    print(f"Error occurred while downloading the PDF for Ticker: {ticker}, Year: {year}")
    continue

        # Save the PDF file


with open(file_path, 'wb') as file:
    file.write(response.content)

Read pdf into dataframes

In [ ]:
def read_pdf(folder_path):
    results = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".pdf"):
            ticker, year = file_name[:-4].split("_")
            file_path = os.path.join(folder_path, file_name)

            try:
                #with open(file_path, 'wb') as file:
                #    file.write(response.content)

                # Read the PDF file using PyPDF2
                with open(file_path, 'rb') as file:
                    reader = PdfReader(file)
                    num_pages = len(reader.pages)

                    # Extract text from each page
                    text = ''
                    for page in reader.pages:
                        text += page.extract_text()
                    #text = text.replace('\n', ' ')  # Replace line breaks with spaces
                    text = re.sub(r'\s+', ' ', text)  # Remove excess space

                    # Extract text from 1st page only
                    text_page1 = reader.pages[0].extract_text()
                    text_page1 = re.sub(r'\s+', ' ', text_page1)


            except Exception as e:
                print(f"Error occurred while processing the PDF file: {file_name}. Skipping file.")
                continue

            # Append results to the list
            results.append({'Ticker': ticker, 'Year': year, 'Filename': file_name, 'Text': text, "Text_page1": text_page1})

    # Create a DataFrame from the results
    output_df = pd.DataFrame(results, columns=['Ticker', 'Year', 'Filename', 'Text', "Text_page1"])
    return output_df

In [ ]:
esg_report_text = read_pdf(path)

In [ ]:
# Export pdf content to excel
esg_report_text.to_excel(r"C:\Users\Hi\OneDrive - University of Bristol\Msc Business Analytics\Wen\esg_report_text.xlsx", index=False)

In [ ]:
#esg_report_text = pd.read_excel(r"C:\Users\Hi\OneDrive - University of Bristol\Msc Business Analytics\Wen\esg_report_text.xlsx")

Manually search for report date and fill in Excel

In [ ]:
esg_report_date = pd.read_excel(r"C:\Users\Hi\OneDrive - University of Bristol\Msc Business Analytics\Wen\esg_report_date.xlsx")
esg_report_date["Report_date"] = esg_report_date["Report_date"].dt.date

Manually check case by case for report titles (automatically extracted from 1st page of report)

In [ ]:
esg_report_name = pd.read_excel(r"C:\Users\Hi\OneDrive - University of Bristol\Msc Business Analytics\Wen\esg_report_name.xlsx")

# 2. Analyse ESG reports

Check if integrate all ESG aspects

In [ ]:
checklist_esg = {
        'Environmental': ['environment', "environmental"],
        'Social': ["social"],
        'Governance': ['governance'],
        'ESG': ['ESG'],
    }

Check if follow any report standard

In [ ]:
checklist_standard = {
        "GRI": ["Global Reporting Initiative", "GRI"],
        "SASB": ["Sustainability Accounting Standards Board", "SASB"],
        "TCFD": ["Task Force on Climate-related Financial Disclosures", "TCFD"],
        "UNSDG": ["United Nations Sustainable Development Goals", "Sustainable Development Goals", "UNSDG", "UN SDG", "UN SDGs", "SDGs"]
    }
keywords_standard = [item for sublist in list(checklist_standard.values()) for item in sublist]

In [ ]:
checklist_all_keywords = {**checklist_esg, **checklist_standard}

In [ ]:
list(checklist_all_keywords)

['Environmental',
 'Social',
 'Governance',
 'ESG',
 'GRI',
 'SASB',
 'TCFD',
 'UNSDG']

In [ ]:
def check_content(text, checklist):
    check_keyword = {}

    for content, keywords in checklist.items():
        for keyword in keywords:
            if re.search(r'\b{}\b'.format(re.escape(keyword)), text, re.IGNORECASE):
                check_keyword[content] = 1
                break
        else:
            check_keyword[content] = 0

    return pd.Series(check_keyword)

Find piece of evidence that the report follows a standard

In [ ]:
def find_sentence(text, keywords):
    sentences = re.split(r'(?<=[.!?])\s+', text)
    text_with_keywords = ""
    for sentence in sentences:
        # Count the number of words in the sentence
        num_words = len(sentence.split())

        # Only show sentences have less than < 40 words to exclude incomplete sentences (eg: all table of contents are counted as 1 sentence)
        if num_words < 40:
            for keyword in keywords:
                if re.search(r'\b{}\b'.format(re.escape(keyword)), sentence, flags=re.IGNORECASE):
                    text_with_keywords += sentence + "\n"
                    break
    return text_with_keywords.strip()

In [ ]:
esg_report_check = esg_report_text.sort_values(["Ticker", "Year"])
esg_report_check["file_accessible"] = np.where(esg_report_check["Text"] == "", 0, 1)
esg_report_check["Independent_report"] = 1
esg_report_check[list(checklist_all_keywords)] = esg_report_check.apply(lambda x: check_content(x["Text"], checklist_all_keywords), axis=1)
esg_report_check["Integrated_report"] = esg_report_check[['Environmental', 'Social', 'Governance',]].min(axis=1)
esg_report_check["standard_notes"] = esg_report_check.apply(lambda x: find_sentence(x["Text"], keywords_standard), axis=1)
esg_report_check["fiscal_year_notes"] = esg_report_check.apply(lambda x: find_sentence(x["Text"], ["fiscal", "calendar"]), axis=1)

In [ ]:
esg_report_check.to_excel(r"C:\Users\Hi\OneDrive - University of Bristol\Msc Business Analytics\Wen\esg_report_check.xlsx", index=False)

In [ ]:
esg_report_check = pd.read_excel(r"C:\Users\Hi\OneDrive - University of Bristol\Msc Business Analytics\Wen\esg_report_check.xlsx")

Export to desired template

In [ ]:
esg_report_analysis = esg_report_link[["Ticker", "Year", "year"]][esg_report_link.year.astype(int) < 2022] \
                        .merge(esg_sample, how="left", left_on="Ticker", right_on="Symbol") \
                        .merge(esg_report_check[["Ticker", "Year", "Independent_report", "Integrated_report",
                                                 "GRI", "SASB", "TCFD", "UNSDG", "file_accessible", "standard_notes"]],
                                                 how="left", on=["Ticker", "Year"]) \
                        .merge(esg_report_date,
                                                how="left", on=["Ticker", "Year"]) \
                        .merge(esg_report_name[["Ticker", "Year", "report_name"]],
                                                how="left", on=["Ticker", "Year"]) \
                        .sort_values(["Ticker", "year"])
esg_report_analysis["Independent_report"].fillna(0, inplace=True)

In [ ]:
writer = pd.ExcelWriter(r"C:\Users\Hi\OneDrive - University of Bristol\Msc Business Analytics\Wen\esg_report_analysis.xlsx", engine='xlsxwriter')

years = esg_report_analysis['year'].unique()

for year in years:
    subset = esg_report_analysis[esg_report_analysis['year'] == year].drop(columns = ["Ticker", "year", "Year"])
    sheet_name = f"overview_{year}"
    subset.to_excel(writer, sheet_name=sheet_name, index=False)

writer.close()